In [10]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
connector = sqlite3.connect('/Users/annaszczepara/Desktop/Code Academy/Project 5/superstore_database.db')

In [11]:
pd.read_sql("Select * from Sales group by ProductID order by Profit DESC", connector)

,SalesID,CustomerID,ProductID,Sales,Quantity,Discount,Profit
0,4833,SC-20095,OFF-BI-10001120,9449.950,5,0.0,4630.4755
1,3153,TA-21385,TEC-CO-10004722,11199.968,4,0.2,3919.9888
2,1348,CM-12385,OFF-BI-10003527,6354.950,5,0.0,3177.4750
3,6372,KD-16270,TEC-MA-10003979,4899.930,7,0.0,2400.9657
4,6473,TB-21400,TEC-MA-10001127,6999.960,4,0.0,2239.9872
...,...,...,...,...,...,...,...
1777,1541,VW-21775,TEC-MA-10000864,1188.000,9,0.7,-950.4000
1778,6499,BM-11140,TEC-MA-10000822,8159.952,8,0.4,-1359.9920
1779,319,TB-21520,FUR-BO-10004834,3083.430,7,0.5,-1665.0522
1780,4185,NF-18385,TEC-MA-10000418,1799.994,2,0.7,-2639.9912


In [4]:
# Plotly-Express

In [5]:
pd.read_sql("Select sum(s.sales) as TotalSales,c.state FROM sales s JOIN customers c ON c.CustomerID = s.CustomerID GROUP BY State ORDER BY TotalSales ASC LIMIT 5", connector)

,TotalSales,State
0,8896.410,Nevada
1,10587.762,Kansas
2,11446.164,Maryland
3,11910.366,Arkansas
4,14035.986,District of Columbia


In [25]:
# How many unique customers per month are there for the year 2016. 
# (There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH()
#  to extract the year or the month in a OrderDate. You will have to create two new columns: year and month.)
pd.read_sql("""Select c.CustomerID, s.OrderMonth, s.OrderYear,
            count(distinct c.customerID) as customer_count
            from shipment s
            join products p on p.OrderID = s.OrderID
            join sales sa on sa.ProductID = p.ProductID
            join customers c on c.CustomerID = sa.CustomerID
            where OrderYear = '2016'
            group by OrderMonth
            """, connector)

,CustomerID,OrderMonth,OrderYear,customer_count
0,BT-11680,1,2016,49
1,CB-12415,2,2016,47
2,AH-10690,3,2016,75
3,CD-12280,4,2016,144
4,LO-17170,5,2016,132
5,BK-11260,6,2016,142
6,CM-12190,7,2016,111
7,AC-10615,8,2016,106
8,AA-10315,9,2016,176
9,HW-14935,10,2016,106
